# Connect 4

In [1]:
import itertools
import numpy as np
import pandas as pd

In [2]:
from platform import python_version
python_version()

'3.7.7'

In [3]:
from connectx import Connect4

In [4]:
game = Connect4()
game.name

'Connect 4'

In [5]:
game.make_move((1, 0))
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
S _ _ _ _ _ _


In [6]:
game._fill_levels

array([1, 0, 0, 0, 0, 0, 0], dtype=int8)

In [7]:
game.make_move((2, 4))
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ _ _ _
S _ _ _ O _ _


In [8]:
game = Connect4()
while not game.game_over:
    role = game.turn
    moves = game.available_moves()
    move = np.random.choice(moves)
    game.make_move((role, move))

game.show_state()
print(game.winner)

_ _ _ _ _ _ _
_ _ _ _ O _ _
_ S _ _ O _ _
_ O _ _ O _ _
_ S _ S O O S
_ O S S S O S
2


In [9]:
game._pos_last

(4, 4)

In [10]:
moves = [
            (1, 3), (2, 6), (1, 0), (2, 5), (1, 2),
            (2, 2), (1, 5), (2, 1), (1, 3), (2, 1),
            (1, 6), (2, 2), (1, 4), (2, 4), (1, 4),
            (2, 4), (1, 1)
        ]
game = Connect4(moves=moves)
game.show_state()

_ _ _ _ _ _ _
_ _ _ _ _ _ _
_ _ _ _ O _ _
_ S O _ S _ _
_ O O S O S S
S O S S S O O


In [11]:
game = Connect4()
pos = (1, 1)  # Note: This is actually (0, 0)
game._get_neighbours(pos)

{'u': 0, 'd': -1, 'r': 0, 'l': -1, 'ur': 0, 'dr': -1, 'ul': -1, 'dl': -1}

In [12]:
STOP!!

SyntaxError: invalid syntax (<ipython-input-12-480d53eab50e>, line 1)

In [ ]:
state[:] = 2
show_state(state)
pos = (1, 1)
for direction in steps.keys():
    print(direction, chain_in_direction(board_full, pos, direction, 2))

In [ ]:

state[:] = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])
move = (2, (1, 1))
assert check_game_over(board_full, move) == False
move = (1, (1, 1))
assert check_game_over(board_full, move) == False

state[:] = np.array([
    [1, 1, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
])
move = (1, (4, 4))
assert check_game_over(board_full, move) == True
move = (1, (4, 3))
assert check_game_over(board_full, move) == True
move = (2, (4, 4))
assert check_game_over(board_full, move) == False
move = (1, (2, 1))
assert check_game_over(board_full, move) == False
move = (1, (3, 2))
assert check_game_over(board_full, move) == False
move = (1, (4, 3))
assert check_game_over(board_full, move) == True
move = (2, (4, 3))
assert check_game_over(board_full, move) == False

state[:] = np.array([
    [1, 1, 1, 0, 0, 0, 0],
    [0, 1, 1, 2, 0, 0, 0],
    [0, 0, 1, 2, 2, 0, 0],
    [0, 0, 2, 0, 2, 0, 0],
    [0, 0, 2, 0, 0, 0, 2],
    [0, 0, 0, 0, 0, 0, 0]
])
move = (2, (4, 4))
assert check_game_over(board_full, move) == False
move = (1, (4, 4))
assert check_game_over(board_full, move) == True
move = (2, (4, 6))
assert check_game_over(board_full, move) == True

In [ ]:
state[:] = np.array([
    [0, 1, 1, 1, 2, 2, 1],
    [0, 0, 1, 0, 2, 1, 0],
    [0, 0, 0, 0, 2, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 1, 0]
])
fill_levels(state)

In [ ]:
# Find winning moves
role = 1
for col in available_cols(state):
    pos = (next_available_position(state, col), col)
    move = (role, (pos[0]+1, pos[1]+1))
    print((role, pos), check_game_over(board_full, move))

role = 2
for col in available_cols(state):
    pos = (next_available_position(state, col), col)
    move = (role, (pos[0]+1, pos[1]+1))
    print((role, pos), check_game_over(board_full, move))

In [ ]:
fl = fill_levels(state)
fl

In [ ]:
size[0]

In [ ]:
fl

In [ ]:
cols = np.arange(1,size[1]+1)
rows = [ for i in range(4)]
rows

In [ ]:
fl

In [ ]:
rows = np.empty((4, size[1]), dtype='int8')
for i in range(4):
    rows[i] = (fl-i).clip(-1,)
rows

In [ ]:
cols = np.empty((4, size[1]), dtype='int8')
cols[:] = np.arange(1,size[1]+1)
cols

In [ ]:
board_full

In [ ]:
board_full[(rows, cols)]

In [ ]:
m_level = int(np.median(fl))
m_level

In [ ]:
state_key = np.empty((5, size[1]), dtype='int8')
state_key[0, :] = (fl - m_level).clip(-1, 2)
state_key[1:, :] = board_full[(rows, cols)]
state_key

### Find all possible (valid) combinations of states for 4x4 segment

In [ ]:
from itertools import product

combinations = np.array(list(product(range(5), repeat=4))).sum(axis=1)
assert len(combinations) == 625

In [ ]:
n_discs = pd.Series(combinations).value_counts().sort_index().rename('No. of boards')
n_discs.index.name = 'No. of discs'
summary = pd.concat([
    n_discs, 
    pd.Series(2**n_discs.index.values, index=n_discs.index, name='Disc combinations')
], axis=1)
summary['Total combinations'] = summary['No. of boards'] * summary['Disc combinations']
summary

In [ ]:
summary['Total combinations'].sum()

### Generate a compact, hashable state representation

In [ ]:
a1 = np.array([64, 16, 4, 1])
a2 = np.array([16777216, 65536, 256, 1])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    assert grid.shape == (4, 4)
    return (np.sum(grid*a1, axis=1)*a2).sum().astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except AssertionError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

In [ ]:
# sum([30.9, 31, 31.1, 28.9])/4 == 30.5
%timeit generate_state_key_uint32(grid)

In [ ]:
a1 = np.array([6, 4, 2, 0])
a2 = np.array([24, 16, 8, 0])

def generate_state_key_uint32(grid):
    """Convert 4x4 int8 array to int64 value.
    """
    assert grid.dtype == 'int8'
    return np.sum(np.sum(grid << a1, axis=1) << a2).astype('uint32')

grid = 3*np.ones((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 2**32-1
assert generate_state_key_uint32(grid).dtype == 'uint32'
assert (generate_state_key_uint32(grid) + 1).dtype == 'int64'
grid = np.zeros((4,4), dtype='int8') 
assert generate_state_key_uint32(grid) == 0
grid[:, 3] = 1
assert generate_state_key_uint32(grid) == 1 + 256 + 256**2 + 256**3
grid = np.ones((4,3), dtype='int8') 
try:
    generate_state_key_uint32(grid)
except ValueError:
    pass

grid = np.random.choice([0, 1, 2], size=16).astype('int8').reshape((4,4))
print(grid)
generate_state_key_uint32(grid)

In [ ]:
# sum([31.2, 31.9, 31.8, 32.4])/4 == 31.8
%timeit generate_state_key_uint32(grid)

In [ ]:
a = np.array([1, 2, 3, 0], dtype='int8')

v = np.sum(a << np.array([6, 4, 2, 0])).astype('uint32')

v

In [ ]:
(v >> 6) & 3, (v >> 4) & 3, (v >> 2) & 3, v & 3

In [ ]:
np.array([v >> 6, v >> 4, v >> 2, v], dtype='int8') & 3